# Training Transformers from Scratch

이번장에서는 언어모델을 학습하기 위한 방법에 대해서 다뤄보도록 하겠습니다. 이번장에서 다루게 될 내용은 크게 다음과 같습니다.

- 매우 큰 사이즈의 데이터를 모으고 정제하는 방법
- 데이터셋에 맞는 커스텀 토크나이저를 만드는 방법
- 모델을 multiple GPUs 환경에서 학습하는 방법

billions 단위의 파라미터를 갖는 모델을 효율적으로 학습하기 위해서는 분산학습을 위한 특별한 도구가 필요합니다. `Transformers`에서 제공하는 `Trainer` 또한 분산학습을 지원하지만, `Accelerate`라는 파워풀한 라이브러리를 소개하면서 진행하도록 하겠습니다. 

In [1]:
!pip install -q transformers[sentencepiece] Datasets Korpora
!apt-get install -y jq

     |████████████████████████████████| 3.4 MB 8.0 MB/s 
     |████████████████████████████████| 306 kB 66.6 MB/s 
     |████████████████████████████████| 57 kB 4.9 MB/s 
     |████████████████████████████████| 132 kB 74.0 MB/s 
     |████████████████████████████████| 61 kB 452 kB/s 
     |████████████████████████████████| 1.1 MB 52.9 MB/s 
     |████████████████████████████████| 243 kB 74.0 MB/s 
     |████████████████████████████████| 96 kB 7.2 MB/s 
     |████████████████████████████████| 271 kB 81.3 MB/s 
     |████████████████████████████████| 160 kB 70.5 MB/s 
     |████████████████████████████████| 192 kB 90.8 MB/s 
     |████████████████████████████████| 895 kB 80.2 MB/s 
     |████████████████████████████████| 3.3 MB 78.6 MB/s 
     |████████████████████████████████| 596 kB 80.4 MB/s 
     |████████████████████████████████| 1.2 MB 76.2 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be

# 데이터로딩
실습의 편의를 위해 `Korpora` 에서 제공해주는 데이터셋을 사용하겠습니다.
개발장비의 RAM 용량보다 데이터 크기가 더 큰 경우 대용량 데이터를 로딩하는건 꽤 까다로운 일입니다. 하지만 `Datasets` 라이브러리는 이런 부분을 `memory mapping`과 `streaming`이라는 두가지 기능을 통해 극복했습니다. 

## Memory Mapping
메모리 부족의 한계를 극복하기 위해서 Datasets 라이브러리에서는 파일로 각 데이터셋을 캐싱합니다. Datasets은 RAM에 모든 데이터셋을 올리기보다 read only pointer를 통해 파일을 읽습니다. `delete_extracted=True` 옵션을 통해 필요없어진 파일들은 삭제시킵니다.

In [2]:
# !git clone https://huggingface.co/datasets/transofrmersbook/codeparrot

In [3]:
# from datasets import load_dataset, DownloadConfig

# download_config = DownloadConfig(delete_extracted=True)
# dataset = load_dataset("./codeparrot", split="train",
#                        download_config=download_config)

## Streaming
어떤 데이터셋은 하드드라이브에 저장하는 방법으로는 로딩하기 어려운 경우가 있습니다. 이런 경우엔 line by line으로 데이터를 읽는 방법인 streaming을 사용합니다. 주로 JSON lines, CSV, text 등이 적합합니다.

In [4]:
# streamed_dataset = load_dataset('./codeparrot', split="train", streaming=True)

허깅페이스 허브에 있는 데이터셋 또한 스트리밍으로 접근해서 사용할 수 있습니다.

In [5]:
%%time
import os
from Korpora import Korpora

Korpora.fetch('kcbert') # ~/Korpora에 말뭉치를 내려받습니다.
# Korpora.fetch('korean_petitions') # ~/Korpora에 말뭉치를 내려받습니다.
# !ls -lah $HOME/Korpora/korean_petitions
# !tail -n 3 $HOME/Korpora/korean_petitions/petitions_2017-08

# corpus = Korpora.load("korean_petitions")

[kcbert] download kcbert-train.tar.gzaa: 100%|██████████| 2.10G/2.10G [01:35<00:00, 21.9MB/s]
[kcbert] download kcbert-train.tar.gzab: 100%|██████████| 2.10G/2.10G [02:12<00:00, 15.8MB/s]
[kcbert] download kcbert-train.tar.gzac: 671MB [00:22, 30.3MB/s]


Unzip tar. It needs a few minutes ... done
CPU times: user 15 s, sys: 8.1 s, total: 23.1 s
Wall time: 6min 18s


In [6]:
!head -n 600000 /root/Korpora/kcbert/20190101_20200611_v2.txt > kcbert_corpus.txt

In [7]:
!tail -n 3 kcbert_corpus.txt

한국정부나 일본정부나 둘이 짝짜꿍해서 그런짓을 저질러 놓고 국격은 무슨 얼어죽을 국격..
답도 없고 시끄러울때는 결론이 필요해 --국제사법과 wto 빨리 끌고가서 답내는게 -- 확실히 무릎을 끓던지 --어정쩡하면 계속 피해
그리고 일한협정 싸구려테러집단이뭐냐 시.댕아 아무리 니논리가 맞고 주장에대한 근거가 명확해도 그딴 소리하면 남설득하기힘들다 일본국적이면 이해한다


In [8]:
# !head -n 2 /root/Korpora/korean_petitions/petitions_2017-08

In [9]:
# # jq를 사용한 jsonline 필터링 (참고: https://hbase.tistory.com/167)
# !head -n 2 /root/Korpora/korean_petitions/petitions_2017-08 | jq '.content'

In [10]:
# cat /root/Korpora/korean_petitions/petitions_* | jq '.content' > 'korean_petitions_corpus_large.txt'

In [11]:
# !cat korean_petitions_corpus_large.txt | wc -l

## Building a Tokenizer

In [12]:
# !ls "$HOME/Korpora/korean_petitions"

In [13]:
# from datasets import load_dataset

# dataset = load_dataset('json', data_files='/root/Korpora/korean_petitions/petitions_2019*')
# print(dataset)

In [14]:
# dataset['train'][0]

In [15]:
# with open('./korean_petitions_corpus.txt', 'w') as f:
#   for i, content in enumerate(dataset['train']['content']):
#       f.write(content+'\n')

In [16]:
# !cat korean_petitions_corpus.txt | wc -l

참고자료:
- https://gist.github.com/lovit/e11c57877aae4286ade4c203d6c26a32
- https://velog.io/@nawnoes/Huggingface-tokenizers%EB%A5%BC-%EC%82%AC%EC%9A%A9%ED%95%9C-Wordpiece-Tokenizer-%EB%A7%8C%EB%93%A4%EA%B8%B0
- https://monologg.kr/2020/04/27/wordpiece-vocab/

In [17]:
from tokenizers import (ByteLevelBPETokenizer,
                        CharBPETokenizer,
                        SentencePieceBPETokenizer,
                        BertWordPieceTokenizer)

# small_corpus = 'korean_petitions_corpus_large.txt'
small_corpus = 'kcbert_corpus.txt'

## Bert WordPiece Tokenizer

In [18]:
%%time 

# 주의해야할 점은 lowercase=False로 할 시 strip_accent=False로 해줘야 한다는 것!
# [UNK]의 비중을 최대한 줄이기 위해 모든 character를 커버할 수 있도록 처리! (limit_alphabet)
bert_wordpiece_tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=False, # Must be False if cased model
    lowercase=False,
    wordpieces_prefix="##"
)
bert_wordpiece_tokenizer.train(
    files = [small_corpus],
    vocab_size = 32000,
    # min_frequency = 3,
    # limit_alphabet = 6000,
    initial_alphabet = [],
    special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    show_progress = True,    
)
vocab = bert_wordpiece_tokenizer.get_vocab()
print(sorted(vocab, key=lambda x: vocab[x]))

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', '!', '"', '%', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'S', 'T', 'V', 'W', 'X', 'Y', '^', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', '~', 'ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅎ', 'ㅏ', 'ㅐ', 'ㅗ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', 'ㆍ', '가', '각', '간', '갈', '감', '갑', '값', '갓', '갔', '강', '갖', '같', '개', '객', '갱', '걍', '거', '걱', '건', '걷', '걸', '검', '겁', '것', '겄', '겉', '게', '겐', '겟', '겠', '겨', '격', '겪', '견', '결', '겹', '겼', '경', '계', '고', '곡', '곤', '곧', '골', '곳', '공', '과', '관', '광', '괜', '괴', '교', '구', '국', '군', '굳', '굴', '굶', '굽', '궁', '권', '귀', '규', '균', '그', '극', '근', '글', '금', '급', '긋', '긍', '기', '긴', '길', '김', '깊', '까', '깍', '깐', '깔', '깜', '깝', '깡', '깨', '깽', '꺼', '껀', '껄', '껏', '께', '껴', '꼬', '꼭'

In [19]:
sample_text = "허깅페이스로 배우는 자연어처리"
encoding = bert_wordpiece_tokenizer.encode(sample_text)
print(encoding.tokens)
print(encoding.ids)

['[UNK]', '배우', '##는', '자연', '##어', '##처리']
[1, 5104, 1039, 4420, 1123, 5982]


In [20]:
!mkdir BERT
bert_wordpiece_tokenizer.save_model(  
    directory = './BERT',
    prefix = 'bertwordpiece'
)

['./BERT/bertwordpiece-vocab.txt']

In [21]:
# from tokenizers.processors import BertProcessing
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast(vocab_file="./BERT/bertwordpiece-vocab.txt", 
                              do_lower_case=False)
tokenizer.tokenize(sample_text)

['[UNK]', '배우', '##는', '자연', '##어', '##처리']

In [22]:
tokenizer.save_pretrained("bert")

('bert/tokenizer_config.json',
 'bert/special_tokens_map.json',
 'bert/vocab.txt',
 'bert/added_tokens.json',
 'bert/tokenizer.json')

In [23]:
bert_wordpiece_tokenizer = BertWordPieceTokenizer(
    vocab = './BERT/bertwordpiece-vocab.txt',
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=False, # Must be False if cased model
    lowercase=False,
    wordpieces_prefix="##"
)
bert_wordpiece_tokenizer.encode('허깅페이스로 배우는 자연어처리', 
                                add_special_tokens=False).tokens

['[UNK]', '배우', '##는', '자연', '##어', '##처리']

In [24]:
from transformers import AutoConfig, AutoModelForMaskedLM, AutoModelForCausalLM, AutoTokenizer
from transformers import BertConfig
# tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
# config = AutoConfig.from_pretrained("gpt2-xl", vocab_size=len(tokenizer))

config = BertConfig(vocab_size=len(tokenizer),
                    num_hidden_layers=12,
                    max_position_embeddings=512)
model = AutoModelForMaskedLM.from_config(config)

In [25]:
model.num_parameters()

110650880

In [26]:
def model_size(model):
    return sum(t.numel() for t in model.parameters())
    
print(f'BERT size: {model_size(model)/1000**2:.1f}M parameters')

BERT size: 110.7M parameters


In [27]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=small_corpus,
    block_size=128,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 1min 4s, sys: 2.24 s, total: 1min 7s
Wall time: 33.2 s


In [28]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [29]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./bert",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [2]:
%%time
trainer.train()

# CPU times: user 5h 52min 26s, sys: 17min 57s, total: 6h 10min 24s
# Wall time: 6h 8min 29s
# TrainOutput(global_step=28125, training_loss=6.99639970703125, metrics={'train_runtime': 22109.1786, 'train_samples_per_second': 81.414, 'train_steps_per_second': 1.272, 'total_flos': 1.0079957514033562e+17, 'train_loss': 6.99639970703125, 'epoch': 3.0})

In [31]:
trainer.save_model("./bert")

Saving model checkpoint to ./bert
Configuration saved in ./bert/config.json
Model weights saved in ./bert/pytorch_model.bin


In [32]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./bert",
    tokenizer="./bert"
)

loading configuration file ./bert/config.json
Model config BertConfig {
  "_name_or_path": "./bert",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading configuration file ./bert/config.json
Model config BertConfig {
  "_name_or_path": "./bert",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_ra

In [33]:
!tail -n 1 $small_corpus

그리고 일한협정 싸구려테러집단이뭐냐 시.댕아 아무리 니논리가 맞고 주장에대한 근거가 명확해도 그딴 소리하면 남설득하기힘들다 일본국적이면 이해한다


In [34]:
fill_mask("안녕하세요 [MASK] ")

[{'score': 0.18943887948989868,
  'sequence': '안녕하세요.',
  'token': 15,
  'token_str': '.'},
 {'score': 0.128323495388031,
  'sequence': '안녕하세요!',
  'token': 5,
  'token_str': '!'},
 {'score': 0.11960893869400024,
  'sequence': '안녕하세요 ~',
  'token': 78,
  'token_str': '~'},
 {'score': 0.02017773501574993,
  'sequence': '안녕하세요 화이팅',
  'token': 3602,
  'token_str': '화이팅'},
 {'score': 0.01876109465956688,
  'sequence': '안녕하세요 ㅋㅋ',
  'token': 1989,
  'token_str': 'ㅋㅋ'}]